In [63]:
import openclsim.apps as apps
import openclsim.model as model
import openclsim.plugins as plugins

import scipy.stats as st
import plotly.graph_objects as go

In [19]:
class DelayBasicActivity(plugins.HasDelayPlugin, model.BasicActivity):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [50]:
class OpenCLSim(apps.SimulationEnvironment):
    
    def define_operation(self):

        # Input property
        DELAY_DIST = st.uniform(0, 2)

        # Base activities
        sail_to = model.BasicActivity(
            env=self.env,
            registry=self.registry,
            name="Sailing to site",
            duration=3600
        )

        position = DelayBasicActivity(
            env=self.env,
            registry=self.registry,
            name="Positioning on site",
            duration=3600 * 2,
            delay_percentage=DELAY_DIST
        )

        # Sequential activity
        seq_activity = model.SequentialActivity(
            env=self.env,
            registry=self.registry,
            name="Sequential activity",
            sub_processes=[sail_to, position]
        )

        # List base activities and store them as a property
        self.activities = [sail_to, position]
        
        # Return upstream activity
        return seq_activity

In [62]:
lengths = []

for i in range(100):
    sim = OpenCLSim(start_date='0')
    sim.initiate_simulation()
    lengths.append(sim.project_length)

In [66]:
go.Figure(data=go.Histogram(
    x=lengths,
    autobinx=Tru
))